In [60]:
'''import module'''

# basic tool
import os
from functools import reduce
import random

# data process and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# style and text
from matplotlib.font_manager import FontProperties

# fit model and evaluation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [61]:
# working dir
working_path = r"D:\mypython_d\Cathay_Big_data_competition_2019"
os.chdir(working_path)

In [62]:
# use big5 to encoding
train_adj = pd.read_csv("train_cleaned_version1_proc_tree.csv", encoding="Big5", low_memory=False, index_col=0)

In [63]:
train_original = pd.read_csv("train.csv", encoding="Big5", low_memory=False, index_col=0)

In [64]:
test_adj = pd.read_csv("test_cleaned_version1_proc_tree.csv", encoding="Big5", low_memory=False, index_col=0)

In [65]:
test_original = pd.read_csv("test.csv", encoding="Big5", low_memory=False, index_col=0)

# 處理

# 1. AMT系列採用中位數補值(不加總)

In [68]:
amt_list = ["DIEBENEFIT_AMT", "DIEACCIDENT_AMT", "POLICY_VALUE_AMT", "ANNUITY_AMT", 
            "EXPIRATION_AMT", "ACCIDENT_HOSPITAL_REC_AMT", "DISEASES_HOSPITAL_REC_AMT", 
            "OUTPATIENT_SURGERY_AMT", "INPATIENT_SURGERY_AMT", "ILL_ACCELERATION_AMT", 
            "FIRST_CANCER_AMT", "ILL_ADDITIONAL_AMT", "LONG_TERM_CARE_AMT", 
            "MONTHLY_CARE_AMT", "PAY_LIMIT_MED_MISC_AMT"]

In [69]:
for i in amt_list:
    train_original[i].fillna(train_original[i].describe()["50%"], inplace=True)

In [70]:
for i in amt_list:
    train_adj[i] = train_original.loc[train_adj.index][i]

In [71]:
train_adj.drop("TOTAL_AMT", axis=1, inplace=True)

test

In [72]:
for i in amt_list:
    test_original[i].fillna(test_original[i].describe()["50%"], inplace=True)

In [73]:
for i in amt_list:
    test_adj[i] = test_original.loc[test_adj.index][i]

In [74]:
test_adj.drop("TOTAL_AMT", axis=1, inplace=True)

# 2.Feature Engineering: AGE & IF_2ND_GEN_IND(是否為保戶二代(Y/N)):
(1) lowage_is2nd:低年齡者，且是保戶二代(傾向不買重疾險):收入低 + 父母可能幫忙買的作用

(2) midage_isnot2nd:中年齡者，且不是保戶二代(傾向購買重疾險)

(3) mihage_is2nd:中高年齡者，且是保戶二代(傾向購買重疾險)

把AGE & IF_2ND_GEN_IND drop掉

以上幾種類別在數量上皆達一定數量。以上所使用欄位皆無NaN。


In [41]:
train_adj["AGE"].value_counts()

1    27193
2    27090
4    23820
3    21655
Name: AGE, dtype: int64

In [75]:
def lowage_is2nd_engineering(cols):
    
    AGE = cols[0]
    IF_2ND_GEN_IND = cols[1]
    
    if AGE == 1 and IF_2ND_GEN_IND == 1:
        return 1
    else:
        return 0

def midage_isnot2nd_engineering(cols):
    
    AGE = cols[0]
    IF_2ND_GEN_IND = cols[1]
    
    if AGE == 2 and IF_2ND_GEN_IND == 0:
        return 1
    else:
        return 0
    
def mihage_is2nd_engineering(cols):
    
    AGE = cols[0]
    IF_2ND_GEN_IND = cols[1]
    
    if AGE == 3 and IF_2ND_GEN_IND == 1:
        return 1
    else:
        return 0

In [76]:
train_adj["lowage_is2nd"] = train_adj[["AGE", "IF_2ND_GEN_IND"]].apply(lowage_is2nd_engineering, axis=1)
train_adj["midage_isnot2nd"] = train_adj[["AGE", "IF_2ND_GEN_IND"]].apply(midage_isnot2nd_engineering, axis=1)
train_adj["mihage_is2nd"] = train_adj[["AGE", "IF_2ND_GEN_IND"]].apply(mihage_is2nd_engineering, axis=1)

In [77]:
train_adj.drop(["AGE", "IF_2ND_GEN_IND"], axis=1, inplace=True)

test

In [79]:
test_adj["lowage_is2nd"] = test_adj[["AGE", "IF_2ND_GEN_IND"]].apply(lowage_is2nd_engineering, axis=1)
test_adj["midage_isnot2nd"] = test_adj[["AGE", "IF_2ND_GEN_IND"]].apply(midage_isnot2nd_engineering, axis=1)
test_adj["mihage_is2nd"] = test_adj[["AGE", "IF_2ND_GEN_IND"]].apply(mihage_is2nd_engineering, axis=1)

In [80]:
test_adj.drop(["AGE", "IF_2ND_GEN_IND"], axis=1, inplace=True)

In [81]:
train_adj.info(max_cols=150)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99758 entries, 3418 to 2512737
Data columns (total 116 columns):
GENDER                       99758 non-null float64
CHARGE_CITY_CD               99758 non-null int64
CONTACT_CITY_CD              99758 non-null int64
EDUCATION_CD                 99758 non-null float64
MARRIAGE_CD                  99758 non-null float64
LAST_A_CCONTACT_DT           99758 non-null int64
L1YR_A_ISSUE_CNT             99758 non-null int64
LAST_A_ISSUE_DT              99758 non-null int64
L1YR_B_ISSUE_CNT             99758 non-null int64
LAST_B_ISSUE_DT              99758 non-null int64
CHANNEL_A_POL_CNT            99758 non-null int64
CHANNEL_B_POL_CNT            99758 non-null int64
OCCUPATION_CLASS_CD          99758 non-null float64
APC_CNT                      99758 non-null int64
INSD_CNT                     99758 non-null int64
APC_1ST_AGE                  99758 non-null float64
INSD_1ST_AGE                 99758 non-null float64
APC_1ST_YEARDIF        

In [82]:
test_adj.info(max_cols=150)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1193 to 2558094
Data columns (total 115 columns):
GENDER                       150000 non-null float64
CHARGE_CITY_CD               150000 non-null int64
CONTACT_CITY_CD              150000 non-null int64
EDUCATION_CD                 150000 non-null float64
MARRIAGE_CD                  150000 non-null float64
LAST_A_CCONTACT_DT           150000 non-null int64
L1YR_A_ISSUE_CNT             150000 non-null int64
LAST_A_ISSUE_DT              150000 non-null int64
L1YR_B_ISSUE_CNT             150000 non-null int64
LAST_B_ISSUE_DT              150000 non-null int64
CHANNEL_A_POL_CNT            150000 non-null int64
CHANNEL_B_POL_CNT            150000 non-null int64
OCCUPATION_CLASS_CD          150000 non-null float64
APC_CNT                      150000 non-null int64
INSD_CNT                     150000 non-null int64
APC_1ST_AGE                  150000 non-null float64
INSD_1ST_AGE                 150000 non-null float64
APC_1

In [83]:
train_adj.to_csv("train_cleaned_version2_proc_tree.csv")
test_adj.to_csv("test_cleaned_version2_proc_tree.csv")

# One hot encoding

In [84]:
A = pd.get_dummies(train_adj["CHARGE_CITY_CD"], drop_first=True)
B = pd.get_dummies(train_adj["CONTACT_CITY_CD"], drop_first=True)
C = pd.get_dummies(train_adj["MARRIAGE_CD"], drop_first=True)
D = pd.get_dummies(train_adj["CUST_9_SEGMENTS_CD"], drop_first=True)
E = pd.get_dummies(train_adj["APC_1ST_AGE"], drop_first=True)
F = pd.get_dummies(train_adj["REBUY_TIMES_CNT"], drop_first=True)
G = pd.get_dummies(train_adj["RFM_M_LEVEL"], drop_first=True)
H = pd.get_dummies(train_adj["APC_1ST_YEARDIF"], drop_first=True)
I = pd.get_dummies(train_adj["TERMINATION_RATE"], drop_first=True)
J = pd.get_dummies(train_adj["RFM_R"], drop_first=True)
K = pd.get_dummies(train_adj["LEVEL"], drop_first=True)

In [85]:
train_adj_encoding = pd.concat([train_adj, A, B, C, D, E, F, G, H, I, J, K], axis=1)

In [86]:
train_adj_encoding = train_adj_encoding.drop(["CHARGE_CITY_CD", "CONTACT_CITY_CD", "MARRIAGE_CD", 
                      "CUST_9_SEGMENTS_CD", "APC_1ST_AGE", "REBUY_TIMES_CNT", 
                      "RFM_M_LEVEL", "APC_1ST_YEARDIF", "TERMINATION_RATE", "RFM_R", "LEVEL"], axis=1)

In [87]:
train_adj_encoding.to_csv("train_cleaned_version2_proc_nontree.csv")

test

In [88]:
A = pd.get_dummies(test_adj["CHARGE_CITY_CD"], drop_first=True)
B = pd.get_dummies(test_adj["CONTACT_CITY_CD"], drop_first=True)
C = pd.get_dummies(test_adj["MARRIAGE_CD"], drop_first=True)
D = pd.get_dummies(test_adj["CUST_9_SEGMENTS_CD"], drop_first=True)
E = pd.get_dummies(test_adj["APC_1ST_AGE"], drop_first=True)
F = pd.get_dummies(test_adj["REBUY_TIMES_CNT"], drop_first=True)
G = pd.get_dummies(test_adj["RFM_M_LEVEL"], drop_first=True)
H = pd.get_dummies(test_adj["APC_1ST_YEARDIF"], drop_first=True)
I = pd.get_dummies(test_adj["TERMINATION_RATE"], drop_first=True)
J = pd.get_dummies(test_adj["RFM_R"], drop_first=True)
K = pd.get_dummies(test_adj["LEVEL"], drop_first=True)

In [89]:
test_adj_encoding = pd.concat([test_adj, A, B, C, D, E, F, G, H, I, J, K], axis=1)

In [90]:
test_adj_encoding = test_adj_encoding.drop(["CHARGE_CITY_CD", "CONTACT_CITY_CD", "MARRIAGE_CD", 
                      "CUST_9_SEGMENTS_CD", "APC_1ST_AGE", "REBUY_TIMES_CNT", 
                      "RFM_M_LEVEL", "APC_1ST_YEARDIF", "TERMINATION_RATE", "RFM_R", "LEVEL"], axis=1)

In [91]:
test_adj_encoding.to_csv("test_cleaned_version2_proc_nontree.csv")

In [97]:
train_adj_encoding.info(max_cols=160)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99758 entries, 3418 to 2512737
Data columns (total 157 columns):
GENDER                       99758 non-null float64
EDUCATION_CD                 99758 non-null float64
LAST_A_CCONTACT_DT           99758 non-null int64
L1YR_A_ISSUE_CNT             99758 non-null int64
LAST_A_ISSUE_DT              99758 non-null int64
L1YR_B_ISSUE_CNT             99758 non-null int64
LAST_B_ISSUE_DT              99758 non-null int64
CHANNEL_A_POL_CNT            99758 non-null int64
CHANNEL_B_POL_CNT            99758 non-null int64
OCCUPATION_CLASS_CD          99758 non-null float64
APC_CNT                      99758 non-null int64
INSD_CNT                     99758 non-null int64
INSD_1ST_AGE                 99758 non-null float64
LIFE_CNT                     99758 non-null int64
IF_ISSUE_A_IND               99758 non-null int64
IF_ISSUE_B_IND               99758 non-null int64
IF_ISSUE_C_IND               99758 non-null int64
IF_ISSUE_D_IND             

In [96]:
test_adj_encoding.info(max_cols=160)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1193 to 2558094
Data columns (total 156 columns):
GENDER                       150000 non-null float64
EDUCATION_CD                 150000 non-null float64
LAST_A_CCONTACT_DT           150000 non-null int64
L1YR_A_ISSUE_CNT             150000 non-null int64
LAST_A_ISSUE_DT              150000 non-null int64
L1YR_B_ISSUE_CNT             150000 non-null int64
LAST_B_ISSUE_DT              150000 non-null int64
CHANNEL_A_POL_CNT            150000 non-null int64
CHANNEL_B_POL_CNT            150000 non-null int64
OCCUPATION_CLASS_CD          150000 non-null float64
APC_CNT                      150000 non-null int64
INSD_CNT                     150000 non-null int64
INSD_1ST_AGE                 150000 non-null float64
LIFE_CNT                     150000 non-null int64
IF_ISSUE_A_IND               150000 non-null int64
IF_ISSUE_B_IND               150000 non-null int64
IF_ISSUE_C_IND               150000 non-null int64
IF_ISSUE_